In [ ]:
%matplotlib inline
#%matplotlib notebook
from matplotlib import rc
rc('figure', figsize=(15,8))
rc('font', size=12)
rc('text', usetex=False)
rc('image', cmap='viridis')

import healpy as hp
import pickle
from importlib import reload

import qubic.io
from pysimulators import FitsArray
import qubic.fibtools as ft
import qubic.demodulation_lib as dl
import qubic.sb_fitting as sbfit
from qubicpack.utilities import Qubic_DataDir

# Import jchinstrument from ../
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 

global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])

In [ ]:
# INSTRUMENT
ns = 256
d = qubic.qubicdict.qubicDict()
d.read_from_file(global_dir+'/dicts/global_source_oneDet_multiband.dict')

q = qubic.QubicMultibandInstrument(d)



In [ ]:
TESNum = 28

### Old data from JC
dirdata = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/Files/Synthesized Beams/Synthesized_Beams_Files/'
directories = [dirdata + '130GHz-2019-04-18/',
               dirdata + '140GHz-2019-04-10/',
               dirdata + '150GHz-2019-04-06/',
               dirdata + '160GHz-2019-04-08/',
               dirdata + '170GHz-2019-04-14/']
freqs = [130, 140, 150, 160, 170]

### New data from James
# directories = ['/Users/hamilton/Qubic/Calib-TD/SB_FromJames/2020-08-01/',
#                '/Users/hamilton/Qubic/Calib-TD/SB_FromJames/2020-07-30/',
#                '/Users/hamilton/Qubic/Calib-TD/SB_FromJames/2020-08-02/']
# freqs = [130, 150, 170]

c50 = np.cos(np.radians(50))
azmin = -15./c50
azmax = 15./c50

allmaps = []
allaz = []
allel =[]
for i in range(len(directories)):
    flatmap, az, el = sbfit.get_flatmap(TESNum, directories[i], azmin = azmin, azmax=azmax)
    allmaps.append(flatmap)
    allaz.append(az)
    allel.append(el)

    subplot(1, len(freqs), i+1)
    AZ, EL = np.meshgrid(allaz[i], allel[i])
    AZ = AZ*np.cos(np.deg2rad(50))
    mm, ss = ft.meancut(allmaps[i], 5)
    plt.imshow(allmaps[i], aspect='equal', origin='lower',
               extent=[np.min(AZ), np.max(AZ), np.min(EL), np.max(EL)])
    title('TES {} - {} GHz'.format(TESNum,freqs[i]))

In [ ]:
import matplotlib.animation
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 60  
plt.ion()
fig, ax = plt.subplots()

nframes = len(freqs)

def animate(t):
    plt.cla()
    findex = t
    index = int(floor(findex))
    AZ, EL = np.meshgrid(allaz[i], allel[i])
    AZ = AZ*np.cos(np.deg2rad(50))
    mm, ss = ft.meancut(allmaps[index], 5)
    plt.imshow(allmaps[index], aspect='equal', origin='lower',
               extent=[np.min(AZ), np.max(AZ), np.min(EL), np.max(EL)])
    title('TES {} - {} GHz'.format(TESNum,freqs[index]))
    
matplotlib.animation.FuncAnimation(fig, animate, frames=nframes)

# Trying RGB image

In [ ]:
import scipy.ndimage
TESNum = 95

### Old data from JC
# dirdata = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/Files/Synthesized Beams/Synthesized_Beams_Files/'
# directories = [dirdata + '130GHz-2019-04-18/',
#                dirdata + '140GHz-2019-04-10/',
#                dirdata + '150GHz-2019-04-06/',
#                dirdata + '160GHz-2019-04-08/',
#                dirdata + '170GHz-2019-04-14/']
# freqs = [130, 140, 150, 160, 170]

### New data from James
directories = ['/Users/hamilton/Qubic/Calib-TD/SB_FromJames/2020-08-01/',
               '/Users/hamilton/Qubic/Calib-TD/SB_FromJames/2020-07-30/',
               '/Users/hamilton/Qubic/Calib-TD/SB_FromJames/2020-08-02/']
freqs = [130, 150, 170]

c50 = np.cos(np.radians(50))
azmin = -15./c50
azmax = 15./c50

allmaps = []
allaz = []
allel =[]
for i in range(len(directories)):
    flatmap, az, el = sbfit.get_flatmap(TESNum, directories[i], azmin = azmin, azmax=azmax)
    allmaps.append(flatmap)
    allaz.append(az)
    allel.append(el)
    AZ, EL = np.meshgrid(allaz[i], allel[i])
    AZ = AZ*np.cos(np.deg2rad(50))

import matplotlib.pyplot as plt
from PIL import Image

sh = np.shape(allmaps[0])

import numpy as np
rgbArray = np.zeros((sh[0], sh[1],3), 'uint8')

szfilt = 1.
for i in range(3):
    myimg = scipy.ndimage.gaussian_filter(allmaps[i], szfilt, order=0)
    mm, ss = ft.meancut(myimg, 10)
    mini = np.min(myimg)
    maxi= np.max(myimg)
    mini = 0
    maxi = np.max(myimg)/2.5
    bla = (myimg-mini)/(maxi-mini)*255
    bla [bla >= 255] = 255
    rgbArray[..., i] = (bla-np.min(bla))/(np.max(bla)-np.min(bla))*255
img = Image.fromarray(rgbArray)

figure()
imshow(img, aspect='equal', origin='lower',
               extent=[np.min(AZ), np.max(AZ), np.min(EL), np.max(EL)])
title('TES #{}'.format(TESNum))
xlabel('Azimuth (Degrees)')
ylabel('Elevation (Degrees)')

In [ ]:
from PIL import Image
from matplotlib.backends.backend_pdf import PdfPages

rc('figure', figsize=(16,16))
nperline = 4
nlinesperpage = 4
numline = 0
allimg = []
myfile = '/Users/hamilton/pdfqubic.pdf'
nplot = 1
with PdfPages(myfile) as pp:
    for TESNum in range(1,256):
        allmaps = []
        allaz = []
        allel =[]
        for i in range(len(directories)):
            flatmap, az, el = sbfit.get_flatmap(TESNum, directories[i], azmin = azmin, azmax=azmax)
            allmaps.append(flatmap)
            allaz.append(az)
            allel.append(el)
            AZ, EL = np.meshgrid(allaz[i], allel[i])
            AZ = AZ*np.cos(np.deg2rad(50))


        sh = np.shape(allmaps[0])
        rgbArray = np.zeros((sh[0], sh[1],3), 'uint8')
#         for i in range(3):
#             mini = np.min(allmaps[i])
#             maxi= np.max(allmaps[i])
#             rgbArray[..., i] = (allmaps[i]-mini)/(maxi-mini)*256
#         img = Image.fromarray(rgbArray)
#         allimg.append(img)
        for i in range(3):
            myimg = scipy.ndimage.gaussian_filter(allmaps[i], szfilt, order=0)
            mm, ss = ft.meancut(myimg, 10)
            mini = np.min(myimg)
            maxi= np.max(myimg)
            mini = 0
            maxi = np.max(myimg)/2.5
            bla = (myimg-mini)/(maxi-mini)*255
            bla [bla >= 255] = 255
            rgbArray[..., i] = (bla-np.min(bla))/(np.max(bla)-np.min(bla))*255
        img = Image.fromarray(rgbArray)

        iline = (TESNum-1) % (nperline * nlinesperpage)
        subplot(nlinesperpage, nperline, nplot)
        imshow(img, aspect='equal', origin='lower',
                   extent=[np.min(AZ), np.max(AZ), np.min(EL), np.max(EL)])
        title('TES #{}'.format(TESNum))
        nplot += 1
        if nplot > ((nperline * nlinesperpage)):
            pp.savefig()
            show()
            figure()
            nplot=1
            

# pp.savefig()
show()